In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegressionCV, LinearRegression, LogisticRegression
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.model_selection import TimeSeriesSplit
from sklearn.experimental import enable_halving_search_cv
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, HalvingRandomSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, mean_squared_error, roc_auc_score
import numpy as np
import joblib

In [2]:
data = pd.read_parquet('./input/creditos_hist.parquet') # Base de la Central de Deudores histórica

In [3]:
# Eliminamos las situaciones 0, que indican que el crédito ya fue pagado
data = data.loc[data['situacion'] != 0]
data = data.drop('denominacion', axis = 1) # Elimino la columna con las razones sociales para ahorrar RAM

In [4]:
data = data.sort_values(by=['identificacion', 'periodo'], ascending= False) # Ordenamos de acuerdo a cada empresa y periodo

In [5]:
# Una variable que puede ser de interés es cuantos créditos tiene una empresa en un momento dado del tiempo
counts = data.groupby(['identificacion', 'periodo']).size().reset_index(name='n_creditos')

# También nos interesa cuanta plata debe una empresa en cada momento dado
sums = data.groupby(['identificacion', 'periodo'], as_index=True)['monto'].sum().reset_index(name='sum_montos')

# La literatura indica que también importa la duración de la relación empresa-banco, por lo que contamos la cantidad 
# de periodos que aparece cada par: empresa-banco
period_counts = data.groupby(['identificacion', 'entidad']).size().reset_index(name='n_periodos')

# Definimos como default cuando el crédito se encuentra en situación 4 o 5, por lo que creamos la dummy de default
# Esta es nuestra variable dependiente
data['default'] = (data['situacion'] >= 4).astype(int)

In [6]:
# Queremos predecir el default el periodo siguiente
data['default_lag'] = data.groupby(['identificacion', 'entidad'])['default'].shift(1) # Lag a la variable default
data = data.dropna(subset=['default_lag']) # Eliminamos las observaciones que no tienen variable dependiente
data['default_lag'] = data['default_lag'].astype(int) # Cambio el dtype de la variable de interés

In [7]:
data['cambio_sit'] = ((data['default'] == 0) & (data['default_lag'] == 1)).astype(int)

In [8]:
data['prev_default'] = (
    data.groupby('identificacion')['default']
    .transform(lambda x: (x.cumsum() > 0).astype(int)) # Si la suma de defaults previa a ese periodo es mayor a 0, quiere decir que la empresa tuvo un default previo
) # Armamos una variable que indique si en algún momento de su historia, esa empresa tuvo un crédito en default

In [9]:
data["sin_historial"] = (
    data.groupby("identificacion")["periodo"]
    .transform("rank", method="first") == 1).astype(int)

# Puede influir el hecho de que la empresa no tenga historial crediticio
# Creamos una variable que indica si es la primera vez que aparece en la base

In [10]:
# Agregamos las nuevas variables al dataframe
data = data.merge(counts, on=['identificacion', 'periodo'], how='left')
data = data.merge(sums, on=['identificacion', 'periodo'], how='left')
data = data.merge(period_counts, on=['identificacion', 'entidad'], how='left')

del sums, counts, period_counts # Para ahorrar RAM

In [11]:
# Por último, la literatura también resalta que la intensidad de la relación empresa-banco es relevante
# Usamos como proxy para la intensidad la proporción del monto adeudado con un banco sobre el total adeudado
data['monto_relativo'] = data['monto'] / data['sum_montos']

In [12]:
data = data.loc[data['periodo'] > '202310']

In [13]:
# Elijo aleatoriamente un porcentaje de las empresas de la población
np.random.seed(42)
cuits = data['identificacion'].unique()
moneda = np.random.binomial(1, 0.2, len(cuits)) # Es como tirar una moneda sesgada para que agarre un porcentaje arbitrario de las empresas 
cuits_aleatorios = cuits[moneda == 1] # Estos son los cuits con los que me voy a quedar
data = data.loc[data['identificacion'].isin(cuits_aleatorios)] # Me quedo unicamente con las obs que tienen un cuit dentro de los seleccionados aleatoriamente

del cuits, moneda, cuits_aleatorios

In [14]:
pv = pd.read_parquet('./input/principales_variables.parquet') # Datos de principales variables monetarias provenientes de la API del BCRA
pv.reset_index(inplace= True) # el index es la fecha, así que lo paso a columna
pv['fecha'] = pd.to_datetime(pv['fecha']) # paso la nueva columna al formato correcto
pv['periodo'] = pv['fecha'].dt.strftime('%Y%m') # armo una variable llamada periodo igual a la que tengo en los datos de la Central de Deudores
pv = pv.drop('fecha', axis = 1).groupby('periodo').agg(['mean', 'std']) # elimino la de "fecha" porque no me interesan los datos diarios
# Me quedo únicamente con los promedios por mes y también calculo el desvío estándar
pv.columns = ['_'.join(col).strip() for col in pv.columns] # Renombro las columnas para que sea más prolijo
pv.reset_index(inplace= True) # Vuelvo a agregar la columna periodo
pv = pv.loc[pv['periodo'].astype(int) <= 202411] # En la Central de Deudores tenemos datos hasta 202410
pv = pv.dropna(axis = 1) # Elimino las columnas con NAs

In [15]:
pv = pv.sort_values("periodo")

pv["inflacion_acumulada"] = (1 + pv["Inflación mensual (variación en %)_mean"]/100).cumprod() # Calculo la inflación acumulada

columnas = pv.columns.tolist()
tasas = [3, 4, 5, 6, 28, 29, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53] # Estos son los índices de las columnas que tienen tasas
excluir = [0, 11, 12, 25, 26, 27, 34, 35, 58, 59, 60, 61, 62, 63, 64] # Estos son los índices de las columnas que ya están en valores reales

columnas_excluir = [columnas[i] for i in excluir]
columnas_tasas = [columnas[i] for i in tasas]

columnas_nominales = [col for col in pv.columns if col not in columnas_excluir and col not in columnas_tasas] # El resto son columnas en valores nominales

for col in columnas_nominales:
    nombre_col_real = f"{col}_real"
    pv[nombre_col_real] = pv[col] / pv['inflacion_acumulada'] # Pasamos los valores de las columnas nominales a valores reales del primer periodo

for col in columnas_tasas:
    nombre_col_real = f"{col}_real"
    pv[col] = pv[col]/100
    pv[nombre_col_real] = pv[col]/pv['inflacion_acumulada'] # A las tasas también las pasamos a valores reales, pero antes las dividimos pro 100
    
    
del col, columnas, columnas_excluir, columnas_tasas, columnas_nominales, tasas, excluir, nombre_col_real

In [16]:
data = data.merge(pv, on = 'periodo', how = 'left') # Junto las principales variables monetarias con la Central de Deudores

In [17]:
data['monto_real'] = data['monto']/data['inflacion_acumulada'] # Pasamos el monto del outstanding a valor real
data['sum_montos_real'] = data['sum_montos']/data['inflacion_acumulada'] # Lo mismo con el monto total adeudado
data.drop('inflacion_acumulada', axis = 1, inplace = True)

In [18]:
emae = pd.read_excel('./input/sh_emae_mensual_base2004.xls', index_col=[0,1]) # EMAE mensual
meses_a_numeros = {
    'Enero': '01', 'Febrero': '02', 'Marzo': '03', 'Abril': '04',
    'Mayo': '05', 'Junio': '06', 'Julio': '07', 'Agosto': '08',
    'Septiembre': '09', 'Octubre': '10', 'Noviembre': '11', 'Diciembre': '12'
} # Diccionario para pasar los meses a números
emae = emae.reset_index()
emae['level_1'] = emae['level_1'].map(meses_a_numeros) # Pasamos los meses a números
emae['periodo'] = emae['Período'].astype(str) + emae['level_1'] # Armamos la variable periodo
emae = emae.drop(columns=['level_1', 'Período'])

del meses_a_numeros

In [19]:
data = data.merge(emae, on = 'periodo', how= 'left') # Juntamos las bases

In [20]:
arca = pd.read_parquet('./input/constancia_inscripcion.parquet') # Cargo los datos de la constancia de inscripción de ARCA

In [21]:
cuits_arca = set(arca['identificacion']) # Los cuits que están en la base de ARCA
cuits_bcra = set(data['identificacion']) # Los cuits que están en la Central de Deudores

faltan = list(cuits_bcra - cuits_arca) # Si están en la Central de Deudores pero no en ARCA es por alguna irregularidad en ARCA

# Creemos que tener irregularidades en ARCA puede ser buen predictor de default
data['sin_arca'] = (data['identificacion'].isin(faltan)).astype(int)  # Creamos la variable

del cuits_arca, cuits_bcra, faltan

In [22]:
# Ponemos bien el tipo de dato para las columnas categóricas, así el get_dummies funciona bien
data['identificacion'] = data['identificacion'].astype('category')
data['entidad'] = data['entidad'].astype('category')
data['situacion'] = data['situacion'].astype('category')
data['default'] = data['default'].astype('category')
data['periodo'] = data['periodo'].astype('category')
data['default_lag'] = data['default_lag'].astype('category')
data['prev_default'] = data['prev_default'].astype('category')
data['sin_arca'] = data['sin_arca'].astype('category')

In [23]:
data.to_parquet('./input/data_limpio.parquet')

Para hacer cross validation, tenemos que tener en cuenta que tenemos un panel. Por lo tanto, vamos a entrenar el modelo con datos del pasado y evaluarlo con datos del futuro

In [24]:
# Cross Validation
data.sort_values(by=['periodo'], ascending= True, inplace = True) # Ordeno de acuerdo a la fecha
data = data.reset_index().drop(columns= 'index')
split_index = int(len(data) * 0.8) # El 80% de las observaciones más antiguas
train_indices = data.iloc[:split_index].index # Estos son los índices con los que después voy a separar en test y train
test_indices = data.iloc[split_index:].index

In [25]:
# Variable dependiente
Y = data['default_lag']

In [26]:
columnas = ['entidad', 'monto', 'n_creditos', 'sum_montos', 'n_periodos', 'monto_relativo', 'sin_arca', 'default', 'prev_default', 'sin_historial', 'monto_real', 'sum_montos_real'] # Algunas de las variables independientes del modelo
# Si agrego efectos fijos por tiempo, "periodo" va después de "sin_historial"
pv.set_index('periodo', inplace= True)
pv.drop('inflacion_acumulada', axis = 1, inplace = True)
columnas.extend(pv.columns) # Todas las columnas de las principales variables monetarias
emae.set_index('periodo', inplace= True)
columnas.extend(emae.columns) # Todas las columnas de emae

In [27]:
#columnas = [col.replace('\n', ' ') for col in columnas]
#with open('./input/columnas.txt', 'w', encoding= 'utf-8') as f:
#    for col in columnas:
#        f.write(col + '\n')

In [28]:
X = pd.get_dummies(data[columnas], drop_first=True) # Meto las columnas en get_dummies

In [29]:
X_cambio_index = X.loc[X['default_1'] != Y].index # Estas son las observaciones que cambiaron de estado entre t y t+1
#X = X.drop('default_1', axis = 1) # Si no queremos que sea autorregresivo droppeamos la variable laggeada

# Separo en entrenamiento y test
X_train = X.loc[train_indices]
Y_train = Y.loc[train_indices]
X_test = X.loc[test_indices]
Y_test = Y.loc[test_indices]

# También identificamos las observaciones en el conjunto de entrenamiento que cambiaron de estado
X_test_cambio_index =list(set(X_test.index) & set(X_cambio_index))
Y_test_cambio_index = list(set(Y_test.index) & set(X_cambio_index))

X_test_cambio = X_test.loc[X_test_cambio_index]
Y_test_cambio = Y_test.loc[Y_test_cambio_index]

del train_indices, test_indices, X_test_cambio_index, Y_test_cambio_index, split_index, X_cambio_index

In [30]:
# Función para evaluar los modelos
def eval(model, X_test, Y_test, linear = None):
    y_pred = model.predict(X_test)
    
    if linear:
        y_pred = np.where(y_pred >= 0.5, 1, 0)
    
    cm = confusion_matrix(Y_test, y_pred)
    
    #precision = precision_score(Y_test, y_pred)
    #recall = recall_score(Y_test, y_pred)
    #f1 = f1_score(Y_test, y_pred)
    accuracy = accuracy_score(Y_test, y_pred)
    mse = mean_squared_error(Y_test, y_pred)
    auc = roc_auc_score(Y_test, y_pred)
    
    print(cm)
    #print(f'La precisión es: {precision}')
    #print(f'El recall es: {recall}')
    #print(f'El f1 es: {f1}')
    print(f'El accuracy es: {accuracy}')
    print(f'El MSE es: {mse}')
    print(f'El AUC es: {auc}')
    
    return y_pred 

In [31]:
# Función para evaluar el MSE promedio y su desvío estándar en los distintos folds para cada lambda
def mse_table(model, elasticnet=None):
    inverse_Cs = 1 / model.Cs_
    results = []
    
    if elasticnet:
        l1_ratios = model.l1_ratios_
        for c_idx, lambda_ in enumerate(inverse_Cs):
            for l1_idx, l1_ratio in enumerate(l1_ratios):
                    mean_score = np.mean(-model.scores_[1][:, c_idx, l1_idx], axis=0)
                    std = np.std(-model.scores_[1][:, c_idx, l1_idx], axis=0)
                    results.append({
                        "Lambda": lambda_,
                        "L1 Ratio": l1_ratio,
                        "Mean MSE": mean_score,
                        "Std MSE": std
                    })
        
        results_table = pd.DataFrame(results).sort_values(by="Mean MSE", ascending=True)
    
    else:
        mean_scores = np.mean(-model.scores_[1], axis=0)
        std = np.std(-model.scores_[1], axis=0)
        results_table = pd.DataFrame({
            "Lambda": inverse_Cs,
            "Mean Score": mean_scores,
            "Std MSE": std
        }).sort_values(by="Mean Score", ascending=True)
    
    print(results_table)
    return results_table

In [32]:
# Función para quedarmos con los coeficientes que son distintos de 0
def non_zero_coefs(model, X_train):
    best_coefs = model.coef_[0]
    feature_names = X_train.columns
    non_zero_coefs = []
    for coef, name in zip(best_coefs, feature_names):
        if coef != 0:
            non_zero_coefs.append({
                "Variable": name,
                "Coeficiente": coef
            })
    non_zero_table = pd.DataFrame(non_zero_coefs)
    print(non_zero_table)
    
    return non_zero_table

## LASSO

In [ ]:
start = -4 
end = 4

tscv = TimeSeriesSplit(n_splits=10) # Este cross validation tiene en cuenta la temporarlidad de la base

pipeline = Pipeline([
    ('scaler', StandardScaler()), # Primero estandariza los datos
    ('logreg', LogisticRegressionCV( # Estima el modelo usando cross validation para elegir el mejor hiperparámetro
        cv=tscv,
        penalty='l1',
        solver='saga',
        scoring='neg_mean_squared_error',
        max_iter=2000,
        random_state= 42,
        tol = 1e-3,
        n_jobs= -1,
        fit_intercept= True,
        Cs= np.logspace(start, end, 20)
    ))
])

del start, end

In [36]:
pipeline.fit(X_train, Y_train) # Entreno el modelo

Pipeline(steps=[('scaler', StandardScaler()),
                ('logreg',
                 LogisticRegressionCV(Cs=array([0.0001    , 0.00011074, 0.00012264, 0.00013582, 0.00015041,
       0.00016657, 0.00018447, 0.00020429, 0.00022624, 0.00025055,
       0.00027747, 0.00030728, 0.0003403 , 0.00037686, 0.00041735,
       0.00046219, 0.00051185, 0.00056684, 0.00062775, 0.00069519]),
                                      cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=10, test_size=None),
                                      max_iter=2000, n_jobs=-1, penalty='l1',
                                      random_state=42,
                                      scoring='neg_mean_squared_error',
                                      solver='saga', tol=0.001))])

In [35]:
joblib.dump(pipeline, './output/lasso_02_arreg.pkl')

['./output/lasso_02_arreg.pkl']

In [43]:
y_pred = eval(pipeline, X_test, Y_test)

[[162058     73]
 [   526  10625]]
El accuracy es: 0.9965432070266964
El MSE es: 0.0034567929733036323
El AUC es: 0.9761895447347647


In [44]:
lasso = pipeline.named_steps['logreg'] # Agarro el modelo desde el pipeline
best_c_lasso = lasso.C_[0]
print(f'El mejor lambda para el modelo es: {1/best_c_lasso}')

El mejor lambda para el modelo es: 3792.6901907322494


In [45]:
mse = mse_table(lasso)

          Lambda  Mean Score   Std MSE
1    3792.690191    0.017235  0.002371
2    1438.449888    0.017235  0.002371
3     545.559478    0.017235  0.002371
4     206.913808    0.017254  0.002412
5      78.475997    0.017578  0.002879
6      29.763514    0.019412  0.005677
7      11.288379    0.019844  0.006459
8       4.281332    0.020111  0.007059
9       1.623777    0.020225  0.007291
10      0.615848    0.020339  0.007571
11      0.233572    0.020428  0.007819
12      0.088587    0.020587  0.008043
13      0.033598    0.020701  0.008325
14      0.012743    0.020904  0.008607
15      0.004833    0.021069  0.008793
16      0.001833    0.021203  0.008957
17      0.000695    0.021412  0.009245
18      0.000264    0.021590  0.009476
19      0.000100    0.021761  0.009732
0   10000.000000    0.043224  0.077802


In [46]:
non_zero_lasso = non_zero_coefs(lasso, X_train)

         Variable  Coeficiente
0      sin_arca_1     0.046724
1       default_1     1.426387
2  prev_default_1     0.914154


## Elastic Net

In [40]:
start = -4
end = 4

tscv = TimeSeriesSplit(n_splits = 5)
pipeline_en = Pipeline([
    ('scaler', StandardScaler()),  
    ('logreg', LogisticRegressionCV(
        cv=tscv,
        penalty='elasticnet',
        solver='saga',
        scoring='neg_mean_squared_error',
        max_iter=2000,
        random_state=42,
        tol=1e-3,
        n_jobs=-1,
        fit_intercept=True,
        Cs=np.logspace(start, end, 20), 
        l1_ratios=np.linspace(0.1, 0.9, 9)  
    ))
])

del start, end

In [41]:
pipeline_en.fit(X_train, Y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('logreg',
                 LogisticRegressionCV(Cs=array([1.00000000e-04, 2.63665090e-04, 6.95192796e-04, 1.83298071e-03,
       4.83293024e-03, 1.27427499e-02, 3.35981829e-02, 8.85866790e-02,
       2.33572147e-01, 6.15848211e-01, 1.62377674e+00, 4.28133240e+00,
       1.12883789e+01, 2.97635144e+01, 7.84759970e+01, 2.06913808e+02,
       5.45559478e+02, 1.43844989e+03, 3.79269019e+03, 1.00000000e+04]),
                                      cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None),
                                      l1_ratios=array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]),
                                      max_iter=2000, n_jobs=-1,
                                      penalty='elasticnet', random_state=42,
                                      scoring='neg_mean_squared_error',
                                      solver='saga', tol=0.001))])

In [42]:
joblib.dump(pipeline_en, './output/elasticnet_02_arreg.pkl')

['./output/elasticnet_02_arreg.pkl']

In [43]:
y_pred = eval(pipeline_en, X_test, Y_test)

[[162058     73]
 [   526  10625]]
El accuracy es: 0.9965432070266964
El MSE es: 0.0034567929733036323
El AUC es: 0.9761895447347647


In [44]:
elasticnet = pipeline_en.named_steps['logreg']
l1_ratio = elasticnet.l1_ratio_[0]
print(f'La proporción óptima de LASSO es: {l1_ratio}')
best_c_en = elasticnet.C_[0]
print(f'El mejor lambda para el modelo es: {1/best_c_en}')

La proporción óptima de LASSO es: 0.6
El mejor lambda para el modelo es: 10000.0


In [45]:
mse = mse_table(elasticnet, True)

         Lambda  L1 Ratio  Mean MSE   Std MSE
17  3792.690191       0.9  0.017271  0.001109
33   545.559478       0.7  0.017271  0.001109
26  1438.449888       0.9  0.017271  0.001109
25  1438.449888       0.8  0.017271  0.001109
24  1438.449888       0.7  0.017271  0.001109
..          ...       ...       ...       ...
81     1.623777       0.1  0.539598  0.667621
72     4.281332       0.1  0.539633  0.667679
63    11.288379       0.1  0.539709  0.667733
54    29.763514       0.1  0.539868  0.667882
45    78.475997       0.1  0.540125  0.668424

[180 rows x 4 columns]


In [46]:
non_zero_en = non_zero_coefs(elasticnet, X_train)

         Variable  Coeficiente
0      sin_arca_1     0.100198
1       default_1     1.182724
2  prev_default_1     0.779892


## RandomForest

In [47]:
rf = RandomForestClassifier()

tscv = TimeSeriesSplit(n_splits = 3)

param_dist = {
    'n_estimators': list(range(100, 701, 10)),  
    'max_depth': [None] + list(range(10, 71, 5)), 
}

halving_search = HalvingRandomSearchCV(
    estimator=rf,
    param_distributions=param_dist,
    factor=3,  
    min_resources=3000,  
    resource="n_samples",  
    cv=tscv,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    random_state=42,
    verbose=4
)

In [48]:
halving_search.fit(X_train, Y_train)

n_iterations: 5
n_required_iterations: 5
n_possible_iterations: 5
min_resources_: 3000
max_resources_: 693125
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 231
n_resources: 3000
Fitting 3 folds for each of 231 candidates, totalling 693 fits
----------
iter: 1
n_candidates: 77
n_resources: 9000
Fitting 3 folds for each of 77 candidates, totalling 231 fits
----------
iter: 2
n_candidates: 26
n_resources: 27000
Fitting 3 folds for each of 26 candidates, totalling 78 fits
----------
iter: 3
n_candidates: 9
n_resources: 81000
Fitting 3 folds for each of 9 candidates, totalling 27 fits
----------
iter: 4
n_candidates: 3
n_resources: 243000
Fitting 3 folds for each of 3 candidates, totalling 9 fits


HalvingRandomSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=3, test_size=None),
                      estimator=RandomForestClassifier(), min_resources=3000,
                      n_jobs=-1,
                      param_distributions={'max_depth': [None, 10, 15, 20, 25,
                                                         30, 35, 40, 45, 50, 55,
                                                         60, 65, 70],
                                           'n_estimators': [100, 110, 120, 130,
                                                            140, 150, 160, 170,
                                                            180, 190, 200, 210,
                                                            220, 230, 240, 250,
                                                            260, 270, 280, 290,
                                                            300, 310, 320, 330,
                                                            340, 350, 360, 370,
                                                            380, 390, ...]},
                      random_state=42, scoring='neg_mean_squared_error',
                      verbose=4)

In [49]:
y_pred = eval(halving_search, X_test, Y_test)

[[162037     94]
 [   474  10677]]
El accuracy es: 0.9967221061622096
El MSE es: 0.0032778938377904226
El AUC es: 0.9784564117398552


In [50]:
joblib.dump(halving_search, './output/rf_02_arreg.pkl')

['./output/rf_02_arreg.pkl']

In [51]:
results_rf = pd.DataFrame(halving_search.cv_results_)

In [52]:
print(halving_search.best_params_)

{'n_estimators': 660, 'max_depth': 35}


In [53]:
rf = halving_search.best_estimator_

In [54]:
importances = pd.DataFrame({
    'Feature': X_train.columns,
    'Importance': rf.feature_importances_
}).sort_values(by='Importance', ascending= False)
print(importances)

                                               Feature    Importance
328                                          default_1  4.555521e-01
329                                     prev_default_1  2.718805e-01
327                                         sin_arca_1  9.192526e-02
3                                           n_periodos  2.245243e-02
7                                      sum_montos_real  1.979750e-02
..                                                 ...           ...
171        entidad_BANCO DE SERVICIOS FINANCIEROS S.A.  3.499027e-13
272  entidad_JPMORGAN CHASE BANK, NATIONAL ASSOCIAT...  0.000000e+00
144  entidad_Asociación Mutual de Pensionados Socia...  0.000000e+00
299                          entidad_POLO VALORES S.A.  0.000000e+00
282                               entidad_MIGXION S.A.  0.000000e+00

[330 rows x 2 columns]


## Modelo de Probabilidad Lineal

In [55]:
pipeline_lm = Pipeline([
    ('scaler', StandardScaler()),
    ('lm', LinearRegression(fit_intercept= True, n_jobs= -1))
                       ])
                    
pipeline_lm.fit(X_train, Y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('lm', LinearRegression(n_jobs=-1))])

In [56]:
joblib.dump(pipeline_lm, './output/lm_02_arreg.pkl')

['./output/lm_02_arreg.pkl']

In [57]:
y_pred = eval(pipeline_lm, X_test, Y_test, linear= True)

[[162058     73]
 [   526  10625]]
El accuracy es: 0.9965432070266964
El MSE es: 0.0034567929733036323
El AUC es: 0.9761895447347647


In [58]:
lm = pipeline_lm.named_steps['lm']

In [59]:
df_coeficientes = pd.DataFrame({
    'variable': X_train.columns,
    'coeficiente': lm.coef_,
    'abs_coef': np.abs(lm.coef_)
}).sort_values(by = 'abs_coef', ascending= False)
print(df_coeficientes.iloc[:,:2])

                                              variable   coeficiente
328                                          default_1  2.189438e-01
329                                     prev_default_1  2.903479e-02
160     entidad_BANCO DE GALICIA Y BUENOS AIRES S.A.U.  7.797441e-03
148                  entidad_BANCO BBVA ARGENTINA S.A.  6.390830e-03
192             entidad_BANCO SANTANDER ARGENTINA S.A.  6.340718e-03
..                                                 ...           ...
110   Tasa de interés de préstamos personales_std_real  1.640974e-07
45         Tasa de Política Monetaria (en % e.a.)_mean  1.363155e-07
144  entidad_Asociación Mutual de Pensionados Socia...  2.710505e-20
282                               entidad_MIGXION S.A.  0.000000e+00
272  entidad_JPMORGAN CHASE BANK, NATIONAL ASSOCIAT...  0.000000e+00

[330 rows x 2 columns]


## Regresión Logística

In [60]:
pipeline_logit = Pipeline([
    ('scaler', StandardScaler()),
	('logit', LogisticRegression(max_iter= 2000, fit_intercept= True, n_jobs = -1, solver = 'saga'))
])
pipeline_logit.fit(X_train, Y_train)

Pipeline(steps=[('scaler', StandardScaler()),
                ('logit',
                 LogisticRegression(max_iter=2000, n_jobs=-1, solver='saga'))])

In [61]:
joblib.dump(pipeline_logit, './output/logit_02_arreg.pkl')

['./output/logit_02_arreg.pkl']

In [62]:
y_pred = eval(pipeline_logit, X_test, Y_test)

[[162057     74]
 [   526  10625]]
El accuracy es: 0.9965374360868411
El MSE es: 0.003462563913158897
El AUC es: 0.9761864608088038


In [63]:
logit = pipeline_logit.named_steps['logit']

In [64]:
df_coeficientes = pd.DataFrame({
    'variable': X_train.columns,
    'coeficiente': logit.coef_[0],
    'abs_coef': np.abs(logit.coef_[0])
}).sort_values(by = 'abs_coef', ascending= False)
print(df_coeficientes.iloc[:, :2])

                                              variable  coeficiente
328                                          default_1     1.575122
329                                     prev_default_1     1.489489
209            entidad_CREDITIA FIDEICOMISO FINANCIERO     0.360975
234                        entidad_F.F PRIVADO YATASTO     0.334818
4                                       monto_relativo     0.272691
..                                                 ...          ...
225                         entidad_Corfa Servi S.R.L.    -0.000110
257                           entidad_GS Negocios S.A.     0.000020
272  entidad_JPMORGAN CHASE BANK, NATIONAL ASSOCIAT...     0.000000
144  entidad_Asociación Mutual de Pensionados Socia...     0.000000
282                               entidad_MIGXION S.A.     0.000000

[330 rows x 2 columns]


In [65]:
#pipeline = joblib.load('./output/lasso_02.pkl')
#pipeline_en = joblib.load('./output/elasticnet_02.pkl')
#random_search = joblib.load('./output/rf_02.pkl')
#pipeline_lm = joblib.load('./output/lm_02.pkl')
#pipeline_logit = joblib.load('./output/logit_02.pkl')

In [66]:
lasso = pipeline.named_steps['logreg']
best_c_lasso = lasso.C_[0]
elasticnet = pipeline_en.named_steps['logreg']
l1_ratio = elasticnet.l1_ratio_[0]
best_c_en = elasticnet.C_[0]
rf = halving_search.best_estimator_
max_depth = rf.max_depth
n_estimators = rf.n_estimators

## VotingClassifier

In [67]:
logit_lasso = LogisticRegression(
    penalty="l1",
    C= best_c_lasso,  
    solver="saga",    
    random_state= 42
)

logit_elastic = LogisticRegression(
    penalty="elasticnet",
    l1_ratio = l1_ratio, 
    C= best_c_en,
    solver="saga",
    random_state= 42
)

logit_no_reg = LogisticRegression(
    random_state= 42
)

rf = RandomForestClassifier(
    n_estimators = n_estimators,
    max_depth= max_depth,
    random_state= 42
)

estimadores = [
    ("logit_lasso", logit_lasso),
    ("logit_elastic", logit_elastic),
    ("logit_no_reg", logit_no_reg),
    ("random_forest", rf)
]

#### Soft

In [68]:
pipeline_voting = Pipeline([
	('scaler', StandardScaler()),
	('voting', VotingClassifier(
     estimators=estimadores,
     voting="soft",
     n_jobs = -1  
))])

In [69]:
tscv = TimeSeriesSplit(n_splits = 5)

param_distributions = {
    'voting__weights': [np.random.dirichlet(np.ones(len(estimadores))) for _ in range(100)]
}

halving_search_vote = HalvingRandomSearchCV(
    estimator=pipeline_voting,
    param_distributions=param_distributions,
    factor=3,  
    min_resources=1000,  
    resource="n_samples",
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    cv=tscv,
    random_state=42,
    verbose=4
)

In [70]:
halving_search_vote.fit(X_train, Y_train)

n_iterations: 5
n_required_iterations: 5
n_possible_iterations: 6
min_resources_: 1000
max_resources_: 693125
aggressive_elimination: False
factor: 3
----------
iter: 0
n_candidates: 100
n_resources: 1000
Fitting 5 folds for each of 100 candidates, totalling 500 fits


c:\Users\HP\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_search.py:317: UserWarning: The total space of parameters 100 is smaller than n_iter=693. Running 100 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


----------
iter: 1
n_candidates: 34
n_resources: 3000
Fitting 5 folds for each of 34 candidates, totalling 170 fits
----------
iter: 2
n_candidates: 12
n_resources: 9000
Fitting 5 folds for each of 12 candidates, totalling 60 fits
----------
iter: 3
n_candidates: 4
n_resources: 27000
Fitting 5 folds for each of 4 candidates, totalling 20 fits
----------
iter: 4
n_candidates: 2
n_resources: 81000
Fitting 5 folds for each of 2 candidates, totalling 10 fits


HalvingRandomSearchCV(cv=TimeSeriesSplit(gap=0, max_train_size=None, n_splits=5, test_size=None),
                      estimator=Pipeline(steps=[('scaler', StandardScaler()),
                                                ('voting',
                                                 VotingClassifier(estimators=[('logit_lasso',
                                                                               LogisticRegression(C=0.00026366508987303583,
                                                                                                  penalty='l1',
                                                                                                  random_state=42,
                                                                                                  solver='saga')),
                                                                              ('logit_elastic',
                                                                               LogisticRegression(C=0.0001,
                                                                                                  l1_ratio=0.6,
                                                                                                  p...
                                                               array([0.08522258, 0.42808073, 0.19788777, 0.28880892]),
                                                               array([0.01188129, 0.04260784, 0.63937376, 0.30613712]),
                                                               array([0.08858762, 0.5835023 , 0.30737116, 0.02053893]),
                                                               array([0.25168105, 0.06137443, 0.04996342, 0.63698109]),
                                                               array([0.43945766, 0.02036807, 0.45344744, 0.08672683]),
                                                               array([0.17232862, 0.06051173, 0.66154505, 0.1056146 ]), ...]},
                      random_state=42, scoring='neg_mean_squared_error',
                      verbose=4)

In [71]:
joblib.dump(halving_search_vote, './output/voting_02_arreg.pkl')

['./output/voting_02_arreg.pkl']

In [72]:
y_pred = eval(halving_search_vote, X_test, Y_test)

[[162057     74]
 [   523  10628]]
El accuracy es: 0.9965547489064069
El MSE es: 0.0034452510935931028
El AUC es: 0.9763209778924734


In [73]:
results_voting = pd.DataFrame(halving_search_vote.cv_results_)

In [74]:
voting = halving_search_vote.best_estimator_

In [75]:
#random_search_vote = joblib.load('./output/voting_02.pkl')

# Sin Autorregresión

| Modelo            | TP    | TN    | FP   | FN  | Accuracy  | MSE       | AUC     |
|------------------|-------|-------|------|-----|-----------|-----------|---------|
| LASSO           | 10603 | 161330 | 801  | 548 | 0.9922    | 0.0078    | 0.9730  |
| Elastic Net     | 10607 | 161329 | 802  | 544 | 0.9922    | 0.0078    | 0.9731  |
| Random Forest   | 10375 | 161734 | 397  | 776 | 0.9932    | 0.0068    | 0.9640  |
| Logit           | 10634 | 161314 | 817  | 517 | 0.9923    | 0.0077    | 0.9743  |
| Linear Reg.     | 10899 | 161018 | 1113 | 252 | 0.9921    | 0.0079    | 0.9853  |
| Voting Classifier | 10673 | 161373 | 758  | 478 | 0.9929    | 0.0071    | 0.9762  |

### Evaluado sobre las observaciones cuyo $default_t \neq default_{t+1}$

| Modelo            | TP  | TN  | FP  | FN  | Accuracy | MSE  | AUC  |
|------------------|----|----|----|----|---------|---------|---------|
| LASSO           | 240 | 12 | 61 | 286 | 0.4207  | 0.5793  | 0.3103  |
| Elastic Net     | 240 | 12 | 61 | 286 | 0.4207  | 0.5793  | 0.3103  |
| Random Forest   | 166 | 11 | 62 | 360 | 0.2955  | 0.7045  | 0.2331  |
| Logit           | 243 | 12 | 61 | 283 | 0.4257  | 0.5743  | 0.3132  |
| Linear Reg.     | 284 | 0  | 73 | 242 | 0.4741  | 0.5259  | 0.2700  |
| Voting Classifier | 235 | 11 | 62 | 291 | 0.4107  | 0.5893  | 0.2987  |

# Con Autorregresión

| Modelo            | TP    | TN    | FP  | FN  | Accuracy  | MSE       | AUC     |
|------------------|-------|-------|-----|-----|-----------|-----------|---------|
| LASSO           | 10625 | 162058 | 73  | 526 | 0.9965    | 0.0035    | 0.9762  |
| Elastic Net     | 10625 | 162058 | 73  | 526 | 0.9965    | 0.0035    | 0.9762  |
| Random Forest   | 10677 | 162037 | 94  | 474 | 0.9967    | 0.0033    | 0.9785  |
| Logit           | 10625 | 162057 | 74  | 526 | 0.9965    | 0.0035    | 0.9762  |
| Linear Reg.     | 10625 | 162058 | 73  | 526 | 0.9965    | 0.0035    | 0.9762  |
| Voting Classifier | 10628 | 162057 | 74  | 523 | 0.9966    | 0.0034    | 0.9763  |

### Evaluado sobre las observaciones cuyo $default_t \neq default_{t+1}$

| Modelo            | TP  | TN  | FP  | FN  | Accuracy  | MSE       | AUC     |
|------------------|----|----|----|----|-----------|-----------|---------|
| LASSO           |  0 |  0 | 73 | 526 | 0.0000    | 1.0000    | 0.0000  |
| Elastic Net     |  0 |  0 | 73 | 526 | 0.0000    | 1.0000    | 0.0000  |
| Random Forest   | 55 |  0 | 73 | 471 | 0.0918    | 0.9082    | 0.0523  |
| Logit           |  0 |  0 | 73 | 526 | 0.0000    | 1.0000    | 0.0000  |
| Linear Reg.     |  0 |  0 | 73 | 526 | 0.0000    | 1.0000    | 0.0000  |
| Voting Classifier |  3 |  0 | 73 | 523 | 0.0050    | 0.9950    | 0.0029  |


In [76]:
print('Cantidad de Créditos que pasaron de estar defaulteados a estar al día:')
print(len(Y_test_cambio.loc[Y_test_cambio == 0]))
print('Cantidad de créditos que pasaron de estar al día a estar defaulteados:')
print(len(Y_test_cambio.loc[Y_test_cambio == 1]))

Cantidad de Créditos que pasaron de estar defaulteados a estar al día:
73
Cantidad de créditos que pasaron de estar al día a estar defaulteados:
526


In [77]:
print('Proporción de créditos al día:')
print(len(Y_test.loc[Y_test == 0])/len(Y_test))

Proporción de créditos al día:
0.9356482496739419


Un predictor trivial que siempre prediga que el crédito no va a estar defaulteado le pegaría el 93.56% de las veces. Para los créditos que en $t$ estaban en un estado y en $t+1$ pasaron a estar en otro, este predictor trivial le pegaría un 12.07% de las veces